In [40]:
import pandas as pd
import math
from IPython.display import display, Latex

#FUNCTIONS 

# covert from degrees to radians
def convert_to_radians(deg):
    return deg * math.pi/180

# Func to format to sig figto_latex
def format_sig_fig(x, sig):
    if isinstance(x, (int, float)):
        if x == 0:  # Check if the value is exactly 0
            return "0"
        return f"{x:.{sig}e}" if abs(x) >= 1e6 or abs(x) < 1e-3 else f"{x:.{sig}g}"
    return x

def to_latex_table(df, caption, label):
    # Build the header row with bold column names
    bold_header = " & ".join([f"\\bfseries {col}" for col in df.columns]) + " \\\\\\midrule"
    
    # Generate the base LaTeX table without the default column headers
    latex_table = df.to_latex(
        index=False,
        header=False,  # Disable default headers to avoid duplication
        escape=False,  # Allow math symbols like ^
        caption=caption,
        label=label,
        bold_rows=False  # Avoid bolding row names
    )
    
    # Replace the top rule and add the custom bold header
    latex_table = latex_table.replace("\\toprule", f"\\toprule\n{bold_header}")
    return latex_table


In [42]:
#DEFINE CONSTANTS
g = 9.81 #m/s

truss_num = 7
beams_num = 14

# Pool Dimensions
pool_l = 16.1                          # length of pool in meters
pool_w = 16.1                          # width of pool in meters
pool_d = 0.24                          # depth of pool in meters

# Glass Modular Dimensions - Laminated Tempered Glass
glass_slab_l = 2.3                         # length of rectangular glass slab in meters
glass_slab_w = 1.15                    # width of rectangular glass slab in meters  
glass_slab_t = 0.06                    # total thickness of glass slab, 60mm

# Densities in (kg/m3) 
rho_water = 1000
rho_temperedGlass =  2520
rho_PVB = 1120    

In [44]:
# MASS CALCULATIONS

Area_pool = pool_l * pool_w                                                            # Area of pool in m

V_water = Area_pool * pool_d                                                  # Volume of water in m3
m_water = round(V_water * rho_water,3)                                              #kg                          

m_glass = round( pool_l * pool_w * glass_slab_t * rho_temperedGlass, 3)   #kg


In [55]:
# WEIGHT CALCULATIONS

W_water = m_water * g
W_glass = m_glass * g

W_total = round(W_water,3) + round(W_glass,3)


In [93]:
# CALCULATIONS FOR EACH FORCE ON TRUSS AND BEAMS

Area_glass = glass_slab_l * glass_slab_w                      # Area of glass slab

F_M = W_total/Area_pool * Area_glass * g                  # Force in each joint [N * m / s^2] = [N] / [m^2] * [m^2] * [m/s^2]

F_B = F_joint/2

By = sum(n * 115 * F_M / pool_l for n in range(1, 14)) + F_B
Ay = By - 14 * F_M 
Ax = 0
F_CE = (F_B + F_M + Ay)/math.sin(convert_to_radians(40))
F_BD = (F_CE * math.sin(convert_to_radians(40)) - F_M)/math.tan(convert_to_radians(40))
F_CF = -Ax - F_BD - F_CE*math.cos(convert_to_radians(40))

# Joint C
F_CB = F_CE
F_CA = F_CF + 2 * F_CE * math.cos(convert_to_radians(40))

# Joint A
F_AB = F_CA / math.cos(convert_to_radians(40))
F_AZ = Ay - F_AB * math.sin(convert_to_radians(40))

In [97]:

# OUTPUT DATA

# Known Measurements
input_data = {
    "Known Measurements": [
        "Pool Length (m)", "Pool Width (m)", "Pool Depth (m)", "Glass Length (m)", "Glass Width (m)", 
        "Glass Thickness (m)", "Density of Water (kg/m$^3$)", "Density of Tempered Glass (kg/m$^3$)", 
        "Pool Surface Area (m$^2$)", "Glass slab Surface Area (m$^2$)"
    ],
    "Value": [
        pool_l, pool_w, pool_d, glass_slab_l, glass_slab_w, 
        glass_slab_t, rho_water, rho_temperedGlass, 
        Area_pool, Area_glass
    ]
}

# Calculated values
calculation_data = {
    "Calculated Forces": [
       "Water Weight", "Tempered Glass Weight", "Total Weight", "Force per Main Joint", "Force per Border Joint"
    ],
    "Value": [
        W_water, W_glass, W_total, F_M, F_joint_end
    ], 
    "Approx": [
        W_water, W_glass, W_total, F_M, F_joint_end
    ]
}

# Force Analysis 
forces_data = {
    "Forces applied in System": [
        "Force per Main Joint", "Force per Border Joint", "Ay", "By", "Ax", "Segment CE (T)", "Segment CF (C)", "Segment BD (T)", "Segment CB (C)", "Segment CA (C)", "Segment AB (T)", "Segment AZ (C)"
    ],
    "Value": [
        F_M, F_joint_end, Ay, By, Ax, F_CE, F_CF, F_BD, F_CB, F_CA, F_AB, F_AZ
    ], 
    "Approx": [
        F_M, F_joint_end, Ay, By, Ax, F_CE, F_CF, F_BD, F_CB, F_CA, F_AB, F_AZ
    ]
}



# Create DataFrames
df_input = pd.DataFrame(input_data)
df_calculations = pd.DataFrame(calculation_data)
df_forces = pd.DataFrame(forces_data)

# apply formatting: 4 sig figs for Value and 3 sig figs for Approx
df_input["Value"] = df_input["Value"].map(lambda x: format_sig_fig(x, sig=4))

df_calculations["Value"] = df_calculations["Value"].map(
    lambda x: f"{format_sig_fig(x, sig=4)} N" if isinstance(x, (int, float)) else x
)

df_calculations["Approx"] = df_calculations["Approx"].map(
    lambda x: (
        f"{format_sig_fig(x / 1e6, sig=3)} MN" if isinstance(x, (int, float)) and x > 1e6 else
        f"{format_sig_fig(x / 1e3, sig=3)} kN" if isinstance(x, (int, float)) and x > 1e3 else
        f"{format_sig_fig(x, sig=3)} N"
    )
)

df_forces["Value"] = df_forces["Value"].map(
    lambda x: f"{format_sig_fig(x, sig=4)} N" if isinstance(x, (int, float)) else x
)

df_forces["Approx"] = df_forces["Approx"].map(
    lambda x: (
        f"{format_sig_fig(x / 1e6, sig=3)} MN" if isinstance(x, (int, float)) and x > 1e6 else
        f"{format_sig_fig(x / 1e3, sig=3)} kN" if isinstance(x, (int, float)) and x > 1e3 else
        f"{format_sig_fig(x, sig=3)} N"
    )
)


# Generate latex tables
latex_input = to_latex_table(df_input, "The measured values of the glass pool and known values of densities.", "known_measurements")

latex_calculations = to_latex_table(df_calculations, "Calculated External Downward Forces on the truss system.", "calculated_forces"
)
latex_forces = to_latex_table(
    df_forces, 
    "External and Internal forces applied in the truss system.", 
    "forces_applied_in_system"
)

# Save latex tables to files
with open("input_table.tex", "w") as file:
    file.write(latex_input)

with open("calculations_table.tex", "w") as file:
    file.write(latex_calculations)

with open("forces_table.tex", "w") as file:
    file.write(latex_forces)
    

# Print or save latex tables
print(latex_input)
print(latex_calculations)
print(latex_forces)

# Display DataFrames in jupyter
display(df_input)
display(df_calculations)
display(df_forces)



\begin{table}
\caption{The measured values of the glass pool and known values of densities.}
\label{known_measurements}
\begin{tabular}{ll}
\toprule
\bfseries Known Measurements & \bfseries Value \\\midrule
\midrule
Pool Length (m) & 16.1 \\
Pool Width (m) & 16.1 \\
Pool Depth (m) & 0.24 \\
Glass Length (m) & 2.3 \\
Glass Width (m) & 1.15 \\
Glass Thickness (m) & 0.06 \\
Density of Water (kg/m$^3$) & 1000 \\
Density of Tempered Glass (kg/m$^3$) & 2520 \\
Pool Surface Area (m$^2$) & 259.2 \\
Glass slab Surface Area (m$^2$) & 2.645 \\
\bottomrule
\end{tabular}
\end{table}

\begin{table}
\caption{Calculated External Downward Forces on the truss system.}
\label{calculated_forces}
\begin{tabular}{lll}
\toprule
\bfseries Calculated Forces & \bfseries Value & \bfseries Approx \\\midrule
\midrule
Water Weight & 6.103e+05 N & 610 kN \\
Tempered Glass Weight & 3.845e+05 N & 384 kN \\
Total Weight & 9.948e+05 N & 995 kN \\
Force per Main Joint & 9.958e+04 N & 99.6 kN \\
Force per Border Joint & 4

,Known Measurements,Value
0,Pool Length (m),16.1
1,Pool Width (m),16.1
2,Pool Depth (m),0.24
3,Glass Length (m),2.3
4,Glass Width (m),1.15
5,Glass Thickness (m),0.06
6,Density of Water (kg/m$^3$),1000
7,Density of Tempered Glass (kg/m$^3$),2520
8,Pool Surface Area (m$^2$),259.2
9,Glass slab Surface Area (m$^2$),2.645


,Calculated Forces,Value,Approx
0,Water Weight,6.103e+05 N,610 kN
1,Tempered Glass Weight,3.845e+05 N,384 kN
2,Total Weight,9.948e+05 N,995 kN
3,Force per Main Joint,9.958e+04 N,99.6 kN
4,Force per Border Joint,4.979e+04 N,49.8 kN


,Forces applied in System,Value,Approx
0,Force per Main Joint,9.958e+04 N,99.6 kN
1,Force per Border Joint,4.979e+04 N,49.8 kN
2,Ay,6.3381e+07 N,63.4 MN
3,By,6.4775e+07 N,64.8 MN
4,Ax,0 N,0 N
5,Segment CE (T),9.8836e+07 N,98.8 MN
6,Segment CF (C),-1.5131e+08 N,-1.513e+08 N
7,Segment BD (T),7.5594e+07 N,75.6 MN
8,Segment CB (C),9.8836e+07 N,98.8 MN
9,Segment CA (C),1.187e+05 N,119 kN
